In [30]:
import pandas as pd
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import KFold, cross_val_score, HalvingGridSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew
from time import perf_counter

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import tensorflow as tf

In [31]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [32]:
root = 'C:/Dev/ELTE_AI/data_mining/comp_1/data/'

In [33]:
# load data and save to pq
#
# train = pd.read_csv(f'{root}/train.csv')
# train.to_parquet(f'{root}/train.parquet')

# test = pd.read_csv(f'{root}/test.csv')
# test.to_parquet(f'{root}/test.parquet')

train = pd.read_parquet(f'{root}/train.parquet')
test = pd.read_parquet(f'{root}/test.parquet')

In [34]:
train.head(20)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,...,0,None,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,None,IR1,Lvl,AllPub,...,0,None,None,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,1,2008,WD,Normal,118000


In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [36]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [37]:
train.drop("Id", axis = 1, inplace = True)
test_IDs = test["Id"]
test.drop("Id", axis = 1, inplace = True)

## EDA and Transforms

In [38]:
train.drop(train[(train['OverallQual']<5) & (train['SalePrice']>200000)].index, inplace=True)
train.drop(train[(train['GrLivArea']>4500) & (train['SalePrice']<300000)].index, inplace=True)
train.reset_index(drop=True, inplace=True)

In [39]:
train["SalePrice"] = np.log1p(train["SalePrice"])

In [40]:
# Split features and labels
y_train = train['SalePrice'].reset_index(drop=True)
train_features = train.drop(['SalePrice'], axis=1)
test_features = test

# Combine train and test features in order to apply the feature transformation pipeline to the entire dataset
all_features = pd.concat([train_features, test_features]).reset_index(drop=True)
all_features.shape

(2916, 79)

In [41]:
# determine the threshold for missing values
def percent_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().mean()*100,2)})

    return dict_x

missing = percent_missing(all_features)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:10]

Percent of missing data


[('PoolQC', 99.69),
 ('MiscFeature', 96.4),
 ('Alley', 93.21),
 ('Fence', 80.42),
 ('FireplaceQu', 48.7),
 ('LotFrontage', 16.63),
 ('GarageYrBlt', 5.45),
 ('GarageFinish', 5.45),
 ('GarageQual', 5.45),
 ('GarageCond', 5.45)]

In [42]:
#

In [43]:
all_features['MSSubClass'] = all_features['MSSubClass'].apply(str)
all_features['YrSold'] = all_features['YrSold'].astype(str)
all_features['MoSold'] = all_features['MoSold'].astype(str)

In [44]:
# the data description states that NA refers to typical ('Typ') values
all_features['Functional'] = all_features['Functional'].fillna('Typ')
all_features['Electrical'] = all_features['Electrical'].fillna("SBrkr")
all_features['KitchenQual'] = all_features['KitchenQual'].fillna("TA")

# the data description stats that NA refers to "No Pool"
all_features["PoolQC"] = all_features["PoolQC"].fillna("None")

# description says nan means no alley access
all_features["Alley"] = all_features["Alley"].fillna("None")

# Replacing the missing values with 0, since no garage = no cars in garage
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_features[col] = all_features[col].fillna(0)

# NaN values for these categorical basement features, means there's no basement
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_features[col] = all_features[col].fillna('None')

# Group the by neighborhoods, and fill in missing value by the median LotFrontage of the neighborhood
all_features['LotFrontage'] = all_features.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

In [45]:
# We have no particular intuition around how to fill in the rest of the categorical features
# So we replace their missing values with None
objects = []
for i in all_features.columns:
    if all_features[i].dtype == object:
        objects.append(i)
all_features.update(all_features[objects].fillna('None'))

# And we do the same thing for numerical features, but this time with 0s
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in all_features.columns:
    if all_features[i].dtype in numeric_dtypes:
        numeric.append(i)
all_features.update(all_features[numeric].fillna(0))

In [46]:
missing = percent_missing(all_features)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:10]

Percent of missing data


[('MSSubClass', 0.0),
 ('MSZoning', 0.0),
 ('LotFrontage', 0.0),
 ('LotArea', 0.0),
 ('Street', 0.0),
 ('Alley', 0.0),
 ('LotShape', 0.0),
 ('LandContour', 0.0),
 ('Utilities', 0.0),
 ('LotConfig', 0.0)]

In [47]:
# Fetch all numeric features
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in all_features.columns:
    if all_features[i].dtype in numeric_dtypes:
        numeric.append(i)

In [48]:
# Find skewed numerical features
skew_features = all_features[numeric].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(10)

There are 25 numerical features with Skew > 0.5 :


MiscVal          21.935910
PoolArea         17.685603
LotArea          13.262550
LowQualFinSF     12.082427
3SsnPorch        11.370087
KitchenAbvGr      4.299698
BsmtFinSF2        4.143683
EnclosedPorch     4.001570
ScreenPorch       3.944305
BsmtHalfBath      3.929197
dtype: float64

In [49]:
# Normalize skewed features
for i in skew_index:
    all_features[i] = boxcox1p(all_features[i], boxcox_normmax(all_features[i] + 1))

C:\Dev\Anaconda\envs\elte_ai\lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
C:\Dev\Anaconda\envs\elte_ai\lib\site-packages\scipy\stats\_stats_py.py:4461: NearConstantInputWarning: An input array is nearly constant; the computed correlation coefficient may be inaccurate.
  warnings.warn(stats.NearConstantInputWarning(msg))


In [50]:
all_features['BsmtFinType1_Unf'] = 1*(all_features['BsmtFinType1'] == 'Unf')
all_features['HasWoodDeck'] = (all_features['WoodDeckSF'] == 0) * 1
all_features['HasOpenPorch'] = (all_features['OpenPorchSF'] == 0) * 1
all_features['HasEnclosedPorch'] = (all_features['EnclosedPorch'] == 0) * 1
all_features['Has3SsnPorch'] = (all_features['3SsnPorch'] == 0) * 1
all_features['HasScreenPorch'] = (all_features['ScreenPorch'] == 0) * 1
all_features['YearsSinceRemodel'] = all_features['YrSold'].astype(int) - all_features['YearRemodAdd'].astype(int)
all_features['Total_Home_Quality'] = all_features['OverallQual'] + all_features['OverallCond']

all_features = all_features.drop(['Utilities', 'Street', 'PoolQC'], axis=1)

all_features['TotalSF'] = all_features['TotalBsmtSF'] + all_features['1stFlrSF'] + all_features['2ndFlrSF']
all_features['YrBltAndRemod'] = all_features['YearBuilt'] + all_features['YearRemodAdd']

all_features['Total_sqr_footage'] = (all_features['BsmtFinSF1'] + all_features['BsmtFinSF2'] +
                                 all_features['1stFlrSF'] + all_features['2ndFlrSF'])
all_features['Total_Bathrooms'] = (all_features['FullBath'] + (0.5 * all_features['HalfBath']) +
                               all_features['BsmtFullBath'] + (0.5 * all_features['BsmtHalfBath']))
all_features['Total_porch_sf'] = (all_features['OpenPorchSF'] + all_features['3SsnPorch'] +
                              all_features['EnclosedPorch'] + all_features['ScreenPorch'] +
                              all_features['WoodDeckSF'])
all_features['TotalBsmtSF'] = all_features['TotalBsmtSF'].apply(lambda x: np.exp(6) if x <= 0.0 else x)
all_features['2ndFlrSF'] = all_features['2ndFlrSF'].apply(lambda x: np.exp(6.5) if x <= 0.0 else x)
all_features['GarageArea'] = all_features['GarageArea'].apply(lambda x: np.exp(6) if x <= 0.0 else x)
all_features['GarageCars'] = all_features['GarageCars'].apply(lambda x: 0 if x <= 0.0 else x)
all_features['LotFrontage'] = all_features['LotFrontage'].apply(lambda x: np.exp(4.2) if x <= 0.0 else x)
all_features['MasVnrArea'] = all_features['MasVnrArea'].apply(lambda x: np.exp(4) if x <= 0.0 else x)
all_features['BsmtFinSF1'] = all_features['BsmtFinSF1'].apply(lambda x: np.exp(6.5) if x <= 0.0 else x)

all_features['haspool'] = all_features['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
all_features['has2ndfloor'] = all_features['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
all_features['hasgarage'] = all_features['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
all_features['hasbsmt'] = all_features['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
all_features['hasfireplace'] = all_features['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [51]:
all_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2916 entries, 0 to 2915
Data columns (total 94 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MSSubClass          2916 non-null   object 
 1   MSZoning            2916 non-null   object 
 2   LotFrontage         2916 non-null   float64
 3   LotArea             2916 non-null   float64
 4   Alley               2916 non-null   object 
 5   LotShape            2916 non-null   object 
 6   LandContour         2916 non-null   object 
 7   LotConfig           2916 non-null   object 
 8   LandSlope           2916 non-null   object 
 9   Neighborhood        2916 non-null   object 
 10  Condition1          2916 non-null   object 
 11  Condition2          2916 non-null   object 
 12  BldgType            2916 non-null   object 
 13  HouseStyle          2916 non-null   object 
 14  OverallQual         2916 non-null   int64  
 15  OverallCond         2916 non-null   float64
 16  YearBu

In [52]:
# descriptors for ordinal
desc = {
    'Alley':{
        'None':0,
        'Grvl':1,
        'Pave':2
    },
    'Utilities':{
        'ELO': 0,
        'NoSeWa': 1,
        'NoSewr': 2,
        'AllPub': 3

    },
    'ExterQual':{
        'Po': 0,
        'Fa': 1,
        'TA': 2,
        'Gd':3,
        'Ex':4
    },
    'ExterCond': {
        'Po': 0,
        'Fa': 1,
        'TA': 2,
        'Gd':3,
        'Ex':4
    },
    'KitchenQual': {
        'Po': 0,
        'Fa': 1,
        'TA': 2,
        'Gd':3,
        'Ex':4
    }
        }

In [53]:
for col in all_features.columns:
    if col in desc.keys():
        all_features[col] = all_features[col].map(desc[col])

In [54]:
all_features = pd.get_dummies(all_features).reset_index(drop=True)

In [55]:
all_features.describe()

,LotFrontage,LotArea,Alley,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_New,SaleType_None,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
count,2916.000000,2916.000000,2916.000000,2916.000000,2916.000000,2916.000000,2916.000000,2916.000000,2916.000000,2916.000000,...,2916.000000,2916.000000,2916.000000,2916.000000,2916.000000,2916.000000,2916.000000,2916.000000,2916.000000,2916.000000
mean,18.672752,14.197857,0.094650,6.087106,4.368443,1971.293896,1984.244856,41.013509,2.395748,2.085734,...,0.081276,0.000343,0.002401,0.865569,0.065158,0.004115,0.008230,0.015775,0.823388,0.083333
std,3.636330,1.152856,0.373145,1.406415,0.761317,30.290493,20.895059,17.405742,0.579024,0.372542,...,0.273305,0.018519,0.048945,0.341173,0.246846,0.064029,0.090363,0.124625,0.381405,0.276433
min,8.722566,10.126989,0.000000,1.000000,0.926307,1872.000000,1950.000000,0.804251,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.239197,13.775927,0.000000,5.000000,3.990331,1953.000000,1965.000000,22.088560,2.000000,2.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,18.999753,14.311164,0.000000,6.000000,3.990331,1973.000000,1993.000000,54.598150,2.000000,2.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,20.660028,14.779245,0.000000,7.000000,4.677969,2001.000000,2004.000000,54.598150,3.000000,2.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,47.727722,22.654942,2.000000,10.000000,6.635013,2010.000000,2010.000000,54.598150,4.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [56]:
all_features

,LotFrontage,LotArea,Alley,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_New,SaleType_None,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,18.133318,14.054262,0,7,3.990331,2003,2003,19.438235,3,2,...,0,0,0,1,0,0,0,0,1,0
1,20.660028,14.347268,0,6,5.997763,1976,1976,54.598150,2,2,...,0,0,0,1,0,0,0,0,1,0
2,18.656316,14.716277,0,7,3.990331,2001,2002,17.773266,3,2,...,0,0,0,1,0,0,0,0,1,0
3,17.239197,14.335210,0,7,3.990331,1915,1970,54.598150,2,2,...,0,0,0,1,1,0,0,0,0,0
4,21.300075,15.277957,0,8,3.990331,2000,2000,25.411674,3,2,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911,8.722566,10.907108,0,4,5.346146,1970,1970,54.598150,2,2,...,0,0,0,1,0,0,0,0,1,0
2912,8.722566,10.863399,0,4,3.990331,1970,1970,54.598150,2,2,...,0,0,0,1,1,0,0,0,0,0
2913,31.730397,16.100709,0,5,5.346146,1960,1996,54.598150,2,2,...,0,0,0,1,1,0,0,0,0,0
2914,17.600363,14.541982,0,5,3.990331,1992,1992,54.598150,2,2,...,0,0,0,1,0,0,0,0,1,0


In [57]:
X_train = all_features.iloc[:len(y_train), :]
X_test = all_features.iloc[len(y_train):, :]
X_train.shape, y_train.shape, X_test.shape

((1457, 333), (1457,), (1459, 333))

## Hyperparameter Optimization using Grid Search

In [58]:
# set the parameter grid
parameters={
            "max_depth": list(np.arange(5, 30, 1)) + [len(all_features.columns)],
            "max_features": list(np.arange(200, 300, 2)) + [len(all_features.columns)],
            "max_leaf_nodes": list(np.arange(70, 120, 2))
            }
# get an estimator object
estimator = DecisionTreeRegressor()

# run optimizer
t_start = perf_counter()
sh_dfl = GridSearchCV(estimator, parameters, cv=5, verbose=1).fit(X_train, y_train)
t_stop = perf_counter()

# reuslts
print(f'Took {(t_stop-t_start)/60:.2f} mins to optimize\nBest params:')
[print(f'{k}: {v}') for k, v in sh_dfl.best_params_.items()]

Fitting 5 folds for each of 33150 candidates, totalling 165750 fits


KeyboardInterrupt: 

In [ ]:
dt = DecisionTreeRegressor(**sh_dfl.best_params_)

In [ ]:
# Setup cross validation folds
kf = KFold(n_splits=25, random_state=999, shuffle=True)
MSE_scorer = make_scorer(mean_squared_error)

In [ ]:
scores = np.sqrt(cross_val_score(dt, X_train, y_train, scoring=MSE_scorer, cv=kf))
print(f'single tree scored {np.mean(scores):.6f} with std of {np.std(scores):.6f}')

In [ ]:
dt_full = dt.fit(X_train, y_train)

In [ ]:
dt_full.get_depth(), dt_full.get_n_leaves()


In [ ]:
Importance = pd.DataFrame(
    {
        'Importance': dt_full.feature_importances_*100
    },
    index=all_features.columns
    )

Importance.sort_values(
    by='Importance',
    axis=0,
    ascending=True
    ).plot(
        kind = 'barh',
        color = 'g',
    )

plt.xlabel('Variable Importance')
plt.gca().legend_ = None

In [ ]:
score = np.sqrt(MSE_scorer(dt_full, X_train, y_train))
print(f'MSE score on train data: {score}')

In [ ]:
preds_test = np.expm1(dt_full.predict(X_test))

In [ ]:
preds_final = pd.DataFrame.from_dict({"Id": test_IDs, "SalePrice": preds_test})
preds_final

In [ ]:
preds_final.to_csv("./data/single_tree_preds_with_hpo.csv", index=None)

In [ ]:
# TODO DL solution

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Input

# normalize
df_norm = X_train.copy()
df_norm['HousePrice'] = y_train
df_norm.iloc[:, :-1] = df_norm.iloc[:, :-1].apply(lambda x: (x-x.mean()) / x.std(), axis=0)

# split
split = .8
split_id = round(split*len(df_norm))
df_norm = df_norm.sample(frac=1, random_state=333).reset_index(drop=True)
df_train = df_norm[:split_id]
df_test = df_norm[split_id:].reset_index(drop=True)

# modeling
inp = Input(shape=(len(df_train.columns)-1))
x = Dense(4, activation='siwsh')(inp)
x = Dense(2, activation='swish')(x)
out = Dense(1, activation='linear')(x)

model = Model(inputs=inp, outputs=out)
model.compile(optimizer='adam', loss=tf.keras.losses.Huber())

# convert dfs to ndarrays
x_train = df_train.iloc[:, :-1].values.astype('float32')
y_train = df_train.iloc[:, -1].values.astype('float32')

x_eval = df_test.iloc[:, :-1].values.astype('float32')
y_eval = df_test.iloc[:, -1].values.astype('float32')

# fit
hist = model.fit(x=x_train,
                 y=y_train,
                 validation_data=(x_eval, y_eval),
                 epochs=100,
                 batch_size=10)

# test
y_preds = model.predict(x_eval, batch_size=10)
y_preds = np.round(y_preds)
